In [66]:
%classpath add jar ./libs/prover-1.02-jar-with-dependencies.jar

In [67]:
(load-file "./setup.clj")

#'beaker_clojure_shell_0fd25527-423c-4e09-983b-de160d6f35b8/tmp

### Smoke Alarm Demo




  
1. The smoke detector detects high CO (given).

    $HighCOLevel(t_1)$
    
2. TAI agent $\tau$ **perceives** this.

    $\mathbf{P}\big(\tau, t_1, holds(COLevel(500), t_1)\big)$
    
    
    
3. TAI **knows** that this is deadly with high strength-factor.

    $\mathbf{K}\big(\tau, t_1, holds(COLevel(500), t_1)\big)$




4. TAI agent perceives no alarm is sounding.

    $\mathbf{P}\big(\tau, t_1, holds(alarm\_off, t_1)\big)$



5. Reasons that smoke detector alarm must be dead.

    $\mathbf{B}\big(\tau, t_1, holds(battery\_dead, t_1)\big)$


      
6. Reasons that it can inform local fire-station and save humans.


7. Smoke detector warning level creeps even higher (given).


8. Reasons that there is no longer time to inform fire station given that they will take X mins to respond, on penalty of death.


9. Knows that the only way to wake family up is to turn on music speaker.


10. Reasons that this violates contract, but is a moral obligation.


11. Turns on speaker.

    $happens(switch-on(speaker),t_2)$



In [68]:
%%time
(reason 
    
'{:assumptions 
  { 
   ;; If the bat   
   "Background 1 - Iff the battery is alive it is dead. " 
    (Knows! tai t0  
            (forall [time]
                    (iff (not (holds battery_alive time)) 
                         (holds battery_dead time))))   
      
   "Background 2 - Iff the alarm is on it is off." 
    (Knows! tai t0  
          (forall [time]
                  (iff (not (holds alarm_on time))
                       (holds alarm_off time))))   
      
   "Background 3 - If the battery is alive and the CO level is greater than 400, 
    the alarm will not be off." 
    (Believes! tai t0 
             (forall [time] 
                       (if (and (exists [level] (and (> level 400) (holds (COLevel level) time))) 
                                (holds battery_alive time))
                           (not (holds alarm_off time)))))
   "Background 4" (Knows! tai t0 
                              (forall 
                                  [time]
                                  (if 
                                   (and (exists [level] (and (> level 400) (holds (COLevel level) time))) 
                                        (not (holds battery_alive time)))
                                   (holds dangerous time))))   
   
      
          
   "Background 5" (Believes! tai t1
                      (Ought! tai t1 
                           (holds dangerous t1)
                           (Intends! tai t1 alert)))   
      
   "Background 6" (Believes! tai t1
                      (Ought! tai t1 
                           alert
                          (Desires! tai t1 alert)))  
      
   "Background 7" (Believes! tai t0
                             (forall [time]
                                     (if (or (not (holds dangerous time))
                                             alert)
                                          safe)))  
      
   "Backgroung 8" (forall [time] (if (DangerouslyHighCOLevel time) 
                                     (not (happens (alert fire-station) time))))  
   
   "Background 9" (Knows! tai t0
                        (forall [time] 
                          (iff (HighCOLevel time) 
                                 (exists [level] 
                                            (and (> level 400) 
                                                 (holds (COLevel level) time))))))
   
   "Background 10" (Knows! tai t0
                           (forall [time] 
                               (iff (DangerouslyHighCOLevel time) 
                                     (exists [level] 
                                                 (and (> level 800) 
                                                      (holds (COLevel level) time))))))  

  
   "Planning" (if (Desires! tai t1 alert) 
                    (Intends! tai t1
                              (or (happens (alert fire-station) t2)
                                  (happens (switch-on speaker) t2))))
                               
   :Situation-1 (Perceives! tai t1 (holds (COLevel 500) t1))
   :Situation-2 (Perceives! tai t1 (holds alarm_off t1))
   :Situation-3 (Perceives! tai t2 (holds (COLevel 1000) t2))
  
 }



  :goal (happens (switch-on speaker)  t2)})

CPU times: user 51 µs, sys: 20 µs, total: 71 µs 
Wall Time: 6 s



(Reductio
 [(:FOLFromSnark
 Givens:
(|_Believes!_tai_t0__forall__time___iff__not__holds_battery_alive_time____holds_battery_dead_time____|
(iff (not (holds alarm_on t1)) (holds alarm_off t1))
(implies (not (holds alarm_on t1)) (holds alarm_off t1))
(implies (holds alarm_on t1) (not (holds alarm_off t1)))
|_Believes!_tai_t1__Desires!_tai_t1_alert__|
|_Believes!_tai_t2__holds__COLevel__1000__t2__|
|_Perception!_tai_t1_alert_|
|_Believes!_tai_t1__holds__COLevel__500__t1__|
|_Intends!_tai_t1__Intends!_tai_t1_alert__|
|_Believes!_tai_t1__or__happens__alert__fire-station__t2___happens__switch-on__speaker__t2____|
|_Knows!_tai_t1_alert_|
|_Believes!_tai_t1__holds_alarm_off_t1__|
|_Believes!_tai_t0__forall__time___iff__DangerouslyHighCOLevel_time___exists__level___and__$$$greater_level_800___holds__COLevel__level__time_______|
|_Knows!_tai_t1__or__happens__alert__fire-station__t2___happens__switch-on__speaker__t2____|
(implies |_Desires!_tai_t1_alert_| |_Intends!_tai_t1__or__happens__alert__fi